Package Install

In [ ]:
! pip install --upgrade numpy==1.21.1
! pip install --upgrade GraphricciCurvature
! pip install pycocotools==2.0.0
! pip3 install pickle5

In [ ]:
import scipy.io as sio
import networkx as nx
import numpy as np
import math
import importlib
import csv
import glob
import sys,pprint
import pandas as pd
import os

# import pickle5 as pickle
import matplotlib.pyplot as plt

import numpy as np
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci import FormanRicci

import networkx.algorithms.community as nx_comm
from sklearn.metrics.cluster import adjusted_rand_score
from google.colab import drive
drive.mount('/content/gdrive')

Function to run ricci flow with multiple rounds.

Here, OllivierRicci is applied. In each round, we choose the subgraph with the most nodes and try to separate this subgraph into several communities structures.

In [1]:
def deep_ricci_flow(G, ori_G, rest_cluster):
    if (rest_cluster == None):
        rest_cluster = []

    if (G.number_of_nodes() < 10):
        return G

    community = nx.connected_components(G)
    origin_cluster = [list(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
    for i in range(len(rest_cluster)):
        origin_cluster.append(rest_cluster[i])
    origin_modularity = nx_comm.modularity(ori_G, origin_cluster)


    cur_G = G.copy()
    edge_weight = []
    num_edge = len(G.edges)
    orc = OllivierRicci(cur_G, alpha=0.5, verbose="ERROR", nbr_topk=20)
    orc.compute_ricci_flow(iterations=15)
    cur_G = orc.G.copy()

    G = cur_G.copy()
    for pair in cur_G.edges:
        edge_weight.append([pair, cur_G[pair[0]][pair[1]]['weight']])
    
    edge_weight = sorted(edge_weight, key=lambda x:x[1], reverse=True)

    j = 0
    gap = int(num_edge*0.05)
    best_modularity = origin_modularity
    best_G = cur_G.copy()
    flag_reduced = False
    cur_num_cluster = 1
    while (j < int(num_edge*0.6)):
        j += 1
        cur_community = nx.connected_components(G)
        cur_cluster = [list(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]

        if (len(cur_cluster) == cur_num_cluster):
            G.remove_edge(edge_weight[j][0][0], edge_weight[j][0][1])
            continue
        else:
            cur_num_cluster = len(cur_cluster)
            for i in range(len(rest_cluster)):
                cur_cluster.append(rest_cluster[i])
            # cur_cluster = cur_cluster + rest_cluster
            cur_modularity = nx_comm.modularity(ori_G, cur_cluster)

            if (cur_modularity > best_modularity):
                best_modularity = cur_modularity
                best_G = G.copy()

            
            # print("update: ", cur_cluster, best_modularity)
        
        if (j / gap == 4) and (len(cur_cluster) == 1):
            best_G = G.copy()
            flag_reduced = True

        G.remove_edge(edge_weight[j][0][0], edge_weight[j][0][1])
    
    best_community = nx.connected_components(best_G)
    best_cluster = [list(c) for c in sorted(best_community, key=len, reverse=True)]
    complete_cluster = [list(c) for c in sorted(nx.connected_components(best_G), key=len, reverse=True)]
    for i in range(len(rest_cluster)):
        complete_cluster.append(rest_cluster[i])
    # print("Complete: ", len(complete_cluster))
    if (len(best_cluster) == 1) and (flag_reduced == False):
        return G
    else:
        new_G = nx.Graph()
        for c in best_cluster:
            # print("iteration: ", len(c))
            temp_cluster = []
            for i in range(len(complete_cluster)):
                if (complete_cluster[i] != c):
                    temp_cluster.append(complete_cluster[i])
            new_sub_G = ori_G.subgraph(c).copy()
            # new_ori_G = ori_G.subgraph(c).copy()
            new_G = nx.union(new_G, deep_ricci_flow(new_sub_G, ori_G, temp_cluster))
        return new_G 

Read the graph files which are Gpickle files.

To compare with the ground truth, we use the modularity to visualize our performance.

In [ ]:
path = "/content/gdrive/MyDrive/Brain_Best_Run_Dataset/SyntheticGpickle/"
files = os.listdir(path)

import pickle5 as pickle

for file in files:
    with open(path+file, "rb") as f:
        G = pickle.load(f)
    print(nx.info(G))
    name = file 
    row = list(ground_truth['Unnamed: 0']).index(name)
    gt_com_num = max((list(ground_truth.loc[row]))[1:])
    gt_com_list = [[] for _ in range (gt_com_num)]
    for i in range(1088):
        gt_com_list[ground_truth.loc[row][i+1]-1].append(i)
    ground_truth_list = [0 for _ in range(1088)]
    for i in range(1088):
        ground_truth_list[i] = ground_truth.loc[row][i+1]

    gt_mod = (nx_comm.modularity(G, gt_com_list))
    print("Modularity of ground truth graph: ", gt_mod) 

    ori_G = G.copy()

    G = deep_ricci_flow(G, ori_G, [])

    community = nx.connected_components(G)
    cluster = [list(c) for c in community]
    print("Final: ", len(cluster))
    print(nx_comm.modularity(ori_G, cluster))
    print(nx.info(G))

    community_index = [0 for _ in range(1088)]
    for i in range(len(cluster)):
        for j in cluster[i]:
            community_index[j] = i
    
    print("ARI: ", adjusted_rand_score(ground_truth_list, community_index))

Store graph file in .mat file.

In [ ]:
path = ""
mat_path = ""

files = os.listdir(path)

for file in files:
    name = file[0:8]
    print(path+file)
    G = nx.read_gpickle(path + file)
    print(nx.info(G))
    break

    cur_com = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    cur_com = sorted(cur_com, key = lambda x: len(x), reverse = True)

    k = len(cur_com)
    member_community = [[0 for _ in range(1088)] for _ in range(k)]

    for i in range(k):
        for j in range(len(cur_com[i])):
            print(cur_com[i])
            member_community[i][cur_com[i][j]] = 1

    sio.savemat(mat_path + name + '_cluster' + '.mat', {'com_members': member_community})